<a href="https://colab.research.google.com/github/Hungtran-pro/Sentiment_Classification_VNese/blob/main/preprocessing_vnese_words.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Using N-gram vs Bayes.

In [ ]:
!pip install pyvi

     |████████████████████████████████| 8.5 MB 15.0 MB/s 
     |████████████████████████████████| 965 kB 3.5 MB/s 


In [ ]:
import re
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.utils import shuffle
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.metrics import f1_score
import string
from pyvi import ViTokenizer
import codecs
import os

In [ ]:
def __getData__(path):
    x_train = []
    y_train = []
    f = open(path, "r")
    raw_data = f.read()

    lst_raw_data = raw_data.split("\n")
    lst_raw_data.pop()
    # print(lst_raw_data[0].split(" ")[0].split("__")[2])
    for sentence in lst_raw_data:
        lst_words = sentence.split(" ")
        # print(" ".join(lst_words[1:]))
        x_train.append(" ".join(lst_words[1:]))
        y_train.append(lst_words[0].split("__")[2])
  
    f.close()      
    return x_train, y_train


In [ ]:
VN_CHARS_LOWER = u'ạảãàáâậầấẩẫăắằặẳẵóòọõỏôộổỗồốơờớợởỡéèẻẹẽêếềệểễúùụủũưựữửừứíìịỉĩýỳỷỵỹđð'
VN_CHARS_UPPER = u'ẠẢÃÀÁÂẬẦẤẨẪĂẮẰẶẲẴÓÒỌÕỎÔỘỔỖỒỐƠỜỚỢỞỠÉÈẺẸẼÊẾỀỆỂỄÚÙỤỦŨƯỰỮỬỪỨÍÌỊỈĨÝỲỶỴỸÐĐ'
VN_CHARS = VN_CHARS_LOWER + VN_CHARS_UPPER

path_nag = '/content/drive/MyDrive/AI projects/Sentiment Classification/VLSP2016_SA/nag.txt'
path_pos = '/content/drive/MyDrive/AI projects/Sentiment Classification/VLSP2016_SA/pos.txt'
path_not = '/content/drive/MyDrive/AI projects/Sentiment Classification/VLSP2016_SA/not.txt'

with codecs.open(path_nag, 'r', encoding='UTF-8') as f:
    nag = f.readlines()
nag_list = [n.replace('\n', '') for n in nag]

with codecs.open(path_pos, 'r', encoding='UTF-8') as f:
    pos = f.readlines()
pos_list = [n.replace('\n', '') for n in pos]
with codecs.open(path_not, 'r', encoding='UTF-8') as f:
    not_ = f.readlines()
not_list = [n.replace('\n', '') for n in not_]

In [ ]:
def no_marks(s):
    __INTAB = [ch for ch in VN_CHARS]
    __OUTTAB = "a"*17 + "o"*17 + "e"*11 + "u"*11 + "i"*5 + "y"*5 + "d"*2
    __OUTTAB += "A"*17 + "O"*17 + "E"*11 + "U"*11 + "I"*5 + "Y"*5 + "D"*2
    __r = re.compile("|".join(__INTAB))
    __replaces_dict = dict(zip(__INTAB, __OUTTAB))
    result = __r.sub(lambda m: __replaces_dict[m.group(0)], s)
    return result

In [ ]:
def normalize_text(text):

    #Remove link
    text = ' '.join([w for w in text.split(" ") if len(w) < 8])
    #Remove các ký tự kéo dài: vd: đẹppppppp
    text = re.sub(r'([A-Z])\1+', lambda m: m.group(1).upper(), text, flags=re.IGNORECASE)

    # Chuyển thành chữ thường
    text = text.lower()

    #Chuẩn hóa tiếng Việt, xử lý emoj, chuẩn hóa tiếng Anh, thuật ngữ
    replace_list = {
        'òa': 'oà', 'óa': 'oá', 'ỏa': 'oả', 'õa': 'oã', 'ọa': 'oạ', 'òe': 'oè', 'óe': 'oé','ỏe': 'oẻ',
        'õe': 'oẽ', 'ọe': 'oẹ', 'ùy': 'uỳ', 'úy': 'uý', 'ủy': 'uỷ', 'ũy': 'uỹ','ụy': 'uỵ', 'uả': 'ủa',
        'ả': 'ả', 'ố': 'ố', 'u´': 'ố','ỗ': 'ỗ', 'ồ': 'ồ', 'ổ': 'ổ', 'ấ': 'ấ', 'ẫ': 'ẫ', 'ẩ': 'ẩ',
        'ầ': 'ầ', 'ỏ': 'ỏ', 'ề': 'ề','ễ': 'ễ', 'ắ': 'ắ', 'ủ': 'ủ', 'ế': 'ế', 'ở': 'ở', 'ỉ': 'ỉ',
        'ẻ': 'ẻ', 'àk': u' à ','aˋ': 'à', 'iˋ': 'ì', 'ă´': 'ắ','ử': 'ử', 'e˜': 'ẽ', 'y˜': 'ỹ', 'a´': 'á',
        #Quy các icon về 2 loại emoj: Tích cực hoặc tiêu cực
        "👹": "nagative", "👻": "positive", "💃": "positive",'🤙': ' positive ', '👍': ' positive ',
        "💄": "positive", "💎": "positive", "💩": "positive","😕": "nagative", "😱": "nagative", "😸": "positive",
        "😾": "nagative", "🚫": "nagative",  "🤬": "nagative","🧚": "positive", "🧡": "positive",'🐶':' positive ',
        '👎': ' nagative ', '😣': ' nagative ','✨': ' positive ', '❣': ' positive ','☀': ' positive ',
        '♥': ' positive ', '🤩': ' positive ', 'like': ' positive ', '💌': ' positive ',
        '🤣': ' positive ', '🖤': ' positive ', '🤤': ' positive ', ':(': ' nagative ', '😢': ' nagative ',
        '❤': ' positive ', '😍': ' positive ', '😘': ' positive ', '😪': ' nagative ', '😊': ' positive ',
        '?': ' ? ', '😁': ' positive ', '💖': ' positive ', '😟': ' nagative ', '😭': ' nagative ',
        '💯': ' positive ', '💗': ' positive ', '♡': ' positive ', '💜': ' positive ', '🤗': ' positive ',
        '^^': ' positive ', '😨': ' nagative ', '☺': ' positive ', '💋': ' positive ', '👌': ' positive ',
        '😖': ' nagative ', '😀': ' positive ', ':((': ' nagative ', '😡': ' nagative ', '😠': ' nagative ',
        '😒': ' nagative ', '🙂': ' positive ', '😏': ' nagative ', '😝': ' positive ', '😄': ' positive ',
        '😙': ' positive ', '😤': ' nagative ', '😎': ' positive ', '😆': ' positive ', '💚': ' positive ',
        '✌': ' positive ', '💕': ' positive ', '😞': ' nagative ', '😓': ' nagative ', '️🆗️': ' positive ',
        '😉': ' positive ', '😂': ' positive ', ':v': '  positive ', '=))': '  positive ', '😋': ' positive ',
        '💓': ' positive ', '😐': ' nagative ', ':3': ' positive ', '😫': ' nagative ', '😥': ' nagative ',
        '😃': ' positive ', '😬': ' 😬 ', '😌': ' 😌 ', '💛': ' positive ', '🤝': ' positive ', '🎈': ' positive ',
        '😗': ' positive ', '🤔': ' nagative ', '😑': ' nagative ', '🔥': ' nagative ', '🙏': ' nagative ',
        '🆗': ' positive ', '😻': ' positive ', '💙': ' positive ', '💟': ' positive ',
        '😚': ' positive ', '❌': ' nagative ', '👏': ' positive ', ';)': ' positive ', '<3': ' positive ',
        '🌝': ' positive ',  '🌷': ' positive ', '🌸': ' positive ', '🌺': ' positive ',
        '🌼': ' positive ', '🍓': ' positive ', '🐅': ' positive ', '🐾': ' positive ', '👉': ' positive ',
        '💐': ' positive ', '💞': ' positive ', '💥': ' positive ', '💪': ' positive ',
        '💰': ' positive ',  '😇': ' positive ', '😛': ' positive ', '😜': ' positive ',
        '🙃': ' positive ', '🤑': ' positive ', '🤪': ' positive ','☹': ' nagative ',  '💀': ' nagative ',
        '😔': ' nagative ', '😧': ' nagative ', '😩': ' nagative ', '😰': ' nagative ', '😳': ' nagative ',
        '😵': ' nagative ', '😶': ' nagative ', '🙁': ' nagative ',
        #Chuẩn hóa 1 số sentiment words/English words
        ':))': '  positive ', ':)': ' positive ', 'ô kêi': ' ok ', 'okie': ' ok ', ' o kê ': ' ok ',
        'okey': ' ok ', 'ôkê': ' ok ', 'oki': ' ok ', ' oke ':  ' ok ',' okay':' ok ','okê':' ok ',
        ' tks ': u' cám ơn ', 'thks': u' cám ơn ', 'thanks': u' cám ơn ', 'ths': u' cám ơn ', 'thank': u' cám ơn ',
        '⭐': 'star ', '*': 'star ', '🌟': 'star ', '🎉': u' positive ',
        'kg ': u' không ','not': u' không ', u' kg ': u' không ', '"k ': u' không ',' kh ':u' không ','kô':u' không ','hok':u' không ',' kp ': u' không phải ',u' kô ': u' không ', '"ko ': u' không ', u' ko ': u' không ', u' k ': u' không ', 'khong': u' không ', u' hok ': u' không ',
        'he he': ' positive ','hehe': ' positive ','hihi': ' positive ', 'haha': ' positive ', 'hjhj': ' positive ',
        ' lol ': ' nagative ',' cc ': ' nagative ','cute': u' dễ thương ','huhu': ' nagative ', ' vs ': u' với ', 'wa': ' quá ', 'wá': u' quá', 'j': u' gì ', '“': ' ',
        ' sz ': u' cỡ ', 'size': u' cỡ ', u' đx ': u' được ', 'dk': u' được ', 'dc': u' được ', 'đk': u' được ',
        'đc': u' được ','authentic': u' chuẩn chính hãng ',u' aut ': u' chuẩn chính hãng ', u' auth ': u' chuẩn chính hãng ', 'thick': u' positive ', 'store': u' cửa hàng ',
        'shop': u' cửa hàng ', 'sp': u' sản phẩm ', 'gud': u' tốt ','god': u' tốt ','wel done':' tốt ', 'good': u' tốt ', 'gút': u' tốt ',
        'sấu': u' xấu ','gut': u' tốt ', u' tot ': u' tốt ', u' nice ': u' tốt ', 'perfect': 'rất tốt', 'bt': u' bình thường ',
        'time': u' thời gian ', 'qá': u' quá ', u' ship ': u' giao hàng ', u' m ': u' mình ', u' mik ': u' mình ',
        'ể': 'ể', 'product': 'sản phẩm', 'quality': 'chất lượng','chat':' chất ', 'excelent': 'hoàn hảo', 'bad': 'tệ','fresh': ' tươi ','sad': ' tệ ',
        'date': u' hạn sử dụng ', 'hsd': u' hạn sử dụng ','quickly': u' nhanh ', 'quick': u' nhanh ','fast': u' nhanh ','delivery': u' giao hàng ',u' síp ': u' giao hàng ',
        'beautiful': u' đẹp tuyệt vời ', u' tl ': u' trả lời ', u' r ': u' rồi ', u' shopE ': u' cửa hàng ',u' order ': u' đặt hàng ',
        'chất lg': u' chất lượng ',u' sd ': u' sử dụng ',u' dt ': u' điện thoại ',u' nt ': u' nhắn tin ',u' tl ': u' trả lời ',u' sài ': u' xài ',u'bjo':u' bao giờ ',
        'thik': u' thích ',u' sop ': u' cửa hàng ', ' fb ': ' facebook ', ' face ': ' facebook ', ' very ': u' rất ',u'quả ng ':u' quảng  ',
        'dep': u' đẹp ',u' xau ': u' xấu ','delicious': u' ngon ', u'hàg': u' hàng ', u'qủa': u' quả ',
        'iu': u' yêu ','fake': u' giả mạo ', 'trl': 'trả lời', '><': u' positive ',
        ' por ': u' tệ ',' poor ': u' tệ ', 'ib':u' nhắn tin ', 'rep':u' trả lời ',u'fback':' feedback ','fedback':' feedback ',
        #dưới 3* quy về 1*, trên 3* quy về 5*
        '6 sao': ' 5star ','6 star': ' 5star ', '5star': ' 5star ','5 sao': ' 5star ','5sao': ' 5star ',
        'starstarstarstarstar': ' 5star ', '1 sao': ' 1star ', '1sao': ' 1star ','2 sao':' 1star ','2sao':' 1star ',
        '2 starstar':' 1star ','1star': ' 1star ', '0 sao': ' 1star ', '0star': ' 1star ',}

    for k, v in replace_list.items():
        text = text.replace(k, v)

    # chuyen punctuation thành space
    translator = str.maketrans(string.punctuation, ' ' * len(string.punctuation))
    text = text.translate(translator)

    text = ViTokenizer.tokenize(text)
    texts = text.split()
    len_text = len(texts)

    texts = [t.replace('_', ' ') for t in texts]
    for i in range(len_text):
        cp_text = texts[i]
        if cp_text in not_list: # Xử lý vấn đề phủ định (VD: áo này chẳng đẹp--> áo này notpos)
            numb_word = 2 if len_text - i - 1 >= 4 else len_text - i - 1

            for j in range(numb_word):
                if texts[i + j + 1] in pos_list:
                    texts[i] = 'notpos'
                    texts[i + j + 1] = ''

                if texts[i + j + 1] in nag_list:
                    texts[i] = 'notnag'
                    texts[i + j + 1] = ''
        else: #Thêm feature cho những sentiment words (áo này đẹp--> áo này đẹp positive)
            if cp_text in pos_list:
                texts.append('positive')
            elif cp_text in nag_list:
                texts.append('nagative')

    text = u' '.join(texts)

    #remove nốt những ký tự thừa thãi
    text = text.replace(u'"', u' ')
    text = text.replace(u'️', u'')
    text = text.replace('🏻','')
    return text

In [ ]:
def __one_hot__(labels):
  for i in range(len(labels)):
    if labels[i] == "POS":
      labels[i] = 2
    elif labels[i] == "NEU":
      labels[i] = 1
    else:
      labels[i] = 0
  return labels

In [ ]:
training_path = "/content/drive/MyDrive/AI projects/Sentiment Classification/VLSP2016_SA/train.txt"
test_path = "/content/drive/MyDrive/AI projects/Sentiment Classification/VLSP2016_SA/test.txt"
# path = (training_path, test_path)
# path = training_path
# print(path)
reviews_train, labels_train = __getData__(training_path)
reviews_test, labels_test = __getData__(test_path)
labels_train = __one_hot__(labels_train)
labels_test = __one_hot__(labels_test)


In [ ]:
#Write to a new file
def __write__(path, contents):
  if os.path.exists(path):
    os.remove(path)
  f = open(path, "a")
  for content in contents:
    f.write(normalize_text(str(content)))
    f.write("\n")
  f.close()
  print(f"Write to file: {path.split('/')[-1]} done!")

In [ ]:
path_filtered_test_X = "/content/drive/MyDrive/AI projects/Sentiment Classification/VLSP2016_SA/filtered_test_X.txt"
path_filtered_test_Y = "/content/drive/MyDrive/AI projects/Sentiment Classification/VLSP2016_SA/filtered_test_Y.txt"
path_filtered_training_X = "/content/drive/MyDrive/AI projects/Sentiment Classification/VLSP2016_SA/filtered_training_X.txt"
path_filtered_training_Y = "/content/drive/MyDrive/AI projects/Sentiment Classification/VLSP2016_SA/filtered_training_Y.txt"
__write__(path_filtered_training_X, reviews_train)
__write__(path_filtered_training_Y, labels_train)
__write__(path_filtered_test_X, reviews_test)
__write__(path_filtered_test_Y, labels_test)

Write to file: filtered_training_X.txt done!
Write to file: filtered_training_Y.txt done!
Write to file: filtered_test_X.txt done!
Write to file: filtered_test_Y.txt done!


#Train and predict using Bayes and N_grams

In [ ]:
def __get_filtered_data__(path, store):
    f = open(path, "r")
    sentences = f.read()
    lst_data = sentences.split("\n")
    lst_data.pop()
    for data in lst_data:
        store.append(data)
    f.close()      
    return store

In [ ]:
filtered_train_X = []
filtered_train_Y = []
filtered_test_X = []
filtered_test_Y = []

filtered_train_X = __get_filtered_data__(path_filtered_training_X, filtered_train_X)
filtered_train_Y = __get_filtered_data__(path_filtered_training_Y, filtered_train_Y)
filtered_test_X = __get_filtered_data__(path_filtered_test_X, filtered_test_X)
filtered_test_Y = __get_filtered_data__(path_filtered_test_Y, filtered_test_Y)
filtered_train_X, filtered_test_X, filtered_train_Y, filtered_test_Y = train_test_split(
    filtered_train_X, filtered_train_Y, test_size = 0.05, random_state = 23)

In [ ]:
for N in range(1,5):
    
    # convert training data to bag of words
    cv = CountVectorizer(analyzer = 'word',ngram_range=(1,N))
    X_train_cv = cv.fit_transform(reviews_train)
    X_test_cv = cv.transform(reviews_test)
    
    # train model and generate predictions
    clf = MultinomialNB()
    clf.fit(X_train_cv, labels_train)
    y_pred = clf.predict(X_test_cv)
    
    # compute f-1 score
    score = np.round(f1_score(labels_test, y_pred, average='micro'),4)
    print('F-1 score of model with n-gram range of {}: {}'.format((1,N), score))

F-1 score of model with n-gram range of (1, 1): 0.6733
F-1 score of model with n-gram range of (1, 2): 0.6867
F-1 score of model with n-gram range of (1, 3): 0.6752
F-1 score of model with n-gram range of (1, 4): 0.6648


In [ ]:
clf.predict(cv.transform(["Quán bình thường, đồ ăn không ngon lắm"]))

array([2])

In [ ]:
sequence_length = 300
embedding_size = 128
num_classes = 3
filter_sizes = 3
num_filters = 150
epochs = 50
batch_size = 30
learning_rate = 0.001
dropout_rate = 0.5

In [ ]:
from gensim.models import Word2Vec
import gensim.models.keyedvectors as word2vec
from tqdm import tqdm
import numpy as np
import os
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers 
import tensorflow as tf
from keras.preprocessing import sequence

In [ ]:
training_path = "/content/drive/MyDrive/AI projects/Sentiment Classification/VLSP2016_SA/train.txt"
test_path = "/content/drive/MyDrive/AI projects/Sentiment Classification/VLSP2016_SA/test.txt"
path = (training_path, test_path)

reviews, labels = __getData__(path)

input_gensim = []
for review in reviews:
    input_gensim.append(review.split())
    
model = Word2Vec(input_gensim, size=128, window=5, min_count=0, workers=4, sg=1)
model.wv.save("word.model")

In [ ]:
model_embedding = word2vec.KeyedVectors.load('./word.model')

word_labels = []
max_seq = 300
embedding_size = 128

for word in model_embedding.vocab.keys():
    word_labels.append(word)
    
def comment_embedding(comment):
    matrix = np.zeros((max_seq, embedding_size))
    words = comment.split()
    lencmt = len(words)

    for i in range(max_seq):
        indexword = i % lencmt
        if (max_seq - i < lencmt):
            break
        if(words[indexword] in word_labels):
            matrix[i] = model_embedding[words[indexword]]
    matrix = np.array(matrix)
    return matrix


In [ ]:
DICT_LABELS = {
    "NEU": [1,0,0],
    "POS": [0,1,0],
    "NEG": [0,0,1]
}

In [ ]:
train_data = []
label_data = []

for x in tqdm(reviews):
    train_data.append(comment_embedding(x))
train_data = np.array(train_data)

for y in tqdm(labels):
  # label_ = np.zeros(3)
  # try:
  #     label_[int(y)] = 1
  # except:
  #     label_[0] = 1
  label_data.append(DICT_LABELS[y])

100%|██████████| 6150/6150 [00:00<00:00, 1916986.44it/s]


In [ ]:
x_train = train_data.reshape(train_data.shape[0], sequence_length, embedding_size, 1).astype('float32')
y_train = np.array(label_data)

# Define model
model = keras.Sequential()
model.add(layers.Convolution2D(num_filters, (filter_sizes, embedding_size),
                        padding='valid',
                        input_shape=(sequence_length, embedding_size, 1), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(198, 1)))
model.add(layers.Dropout(dropout_rate))
model.add(layers.Flatten())
model.add(layers.Dense(126, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))
# Train model
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])
print(model.summary())


In [ ]:
model.fit(x_train[:5100], y_train[:5100], batch_size = batch_size, verbose=1, epochs=epochs, validation_data=(x_train[5100:], y_train[5100:]))

model.save('models.h5')
